### Import required libraries

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import contractor_data_fetcher
#import pca_data_fetcher

### Data sources

#### Contractor data analysis<a href="#contractor-analysis"></a>
Data on claims from community pharmacies for Pharmacy First clinical pathways is published by NHS BSA within the [Pharmacy and appliance contractor dispensing data](https://www.nhsbsa.nhs.uk/prescription-data/dispensing-data/dispensing-contractors-data) - from Feb 24 onwards the dataset contain columns for each clinical condition e.g. "NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia" which contains a count for each community pharmacy.

Here we fetch all csv files from Feb 24 onwards containing this data combine them all then return as a single dataframe:

In [3]:
contractor_df = contractor_data_fetcher.fetch_contractor_data()

Remove unneeded columns:

In [4]:
# Keep only the specified columns
columns_to_keep = [
    'ICBCode', 'ICB', 'NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia', 'NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteSoreThroat',
    'NumberofPharmacyFirstClinicalPathwaysConsultations-Impetigo', 'NumberofPharmacyFirstClinicalPathwaysConsultations-InfectedInsectBites',
    'NumberofPharmacyFirstClinicalPathwaysConsultations-Shingles', 'NumberofPharmacyFirstClinicalPathwaysConsultations-Sinusitis',
    'NumberofPharmacyFirstClinicalPathwaysConsultations-UncomplicatedUTI', 'NumberofPharmacyFirstUrgentMedicineSupplyConsultations',
    'NumberofPharmacyFirstMinorIllnessReferralConsultations', 'month'
]

contractor_df = contractor_df[columns_to_keep]

Add total consultations column:

In [5]:
# Filter columns that start with 'NumberofPharmacyFirstClinicalPathwaysConsultations'
columns_to_sum = contractor_df.filter(regex='^NumberofPharmacyFirstClinicalPathwaysConsultations').columns

# Create the new column by summing the selected columns row-wise
contractor_df['NumberofPharmacyFirstClinicalPathwaysConsultations-Total'] = contractor_df[columns_to_sum].sum(axis=1)

Get national totals:

In [6]:
# Drop the 'ICBCode' and 'ICB' columns to get national view
contractor_df_national = contractor_df.drop(columns=['ICBCode', 'ICB'])

# Group by 'month' and sum the remaining columns
contractor_df_national = contractor_df_national.groupby(['month']).sum().reset_index()

contractor_df_national

,month,NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia,NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteSoreThroat,NumberofPharmacyFirstClinicalPathwaysConsultations-Impetigo,NumberofPharmacyFirstClinicalPathwaysConsultations-InfectedInsectBites,NumberofPharmacyFirstClinicalPathwaysConsultations-Shingles,NumberofPharmacyFirstClinicalPathwaysConsultations-Sinusitis,NumberofPharmacyFirstClinicalPathwaysConsultations-UncomplicatedUTI,NumberofPharmacyFirstUrgentMedicineSupplyConsultations,NumberofPharmacyFirstMinorIllnessReferralConsultations,NumberofPharmacyFirstClinicalPathwaysConsultations-Total
0,2024-02-01,18893,41014,6816,3412,4078,18384,32678,78831,137396,125275
1,2024-03-01,22888,54756,6740,4180,4069,17220,34536,95789,131715,144389
2,2024-04-01,19295,56925,7191,6099,4470,17715,41951,90851,126193,153646
3,2024-05-01,20240,58022,6235,19010,4691,15909,44340,110710,127706,168447
4,2024-06-01,15826,52288,6362,27294,4626,15052,43289,100367,115729,164737
5,2024-07-01,17682,54859,6740,36370,5103,14115,51321,122269,110511,186190


### **TO DO - Compare these against the numbers we get from OpenSAFELY

Get regional totals:

In [7]:
# Group by 'ICBCode', 'ICB', and 'month', then sum the other columns
contractor_df_icb = contractor_df.groupby(['ICBCode', 'ICB', 'month']).sum().reset_index()

#Load region lookup file
icb_to_region_df = pd.read_csv("Sub_ICB_Locations_to_Integrated_Care_Boards_to_NHS_England_(Region)_(July_2022)_Lookup_in_England.csv")

# Keep only the specified columns
icb_to_region_df = icb_to_region_df[['ICB22CDH', 'NHSER22CDH', 'NHSER22NM']]

# Remove duplicates
icb_to_region_df = icb_to_region_df.drop_duplicates()

# Merge region name onto ICB code
contractor_df_icb = contractor_df_icb.merge(icb_to_region_df, left_on='ICBCode', right_on='ICB22CDH', how='left')

# Drop unneeded columns
contractor_df_regional = contractor_df_icb.drop(columns=['ICBCode', 'ICB', 'ICB22CDH', 'NHSER22CDH'])

# Grouping by NHSER22NM and month, summing the remaining columns
contractor_df_regional = contractor_df_regional.groupby(['NHSER22NM', 'month']).sum().reset_index()

contractor_df_regional

,NHSER22NM,month,NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteOtitisMedia,NumberofPharmacyFirstClinicalPathwaysConsultations-AcuteSoreThroat,NumberofPharmacyFirstClinicalPathwaysConsultations-Impetigo,NumberofPharmacyFirstClinicalPathwaysConsultations-InfectedInsectBites,NumberofPharmacyFirstClinicalPathwaysConsultations-Shingles,NumberofPharmacyFirstClinicalPathwaysConsultations-Sinusitis,NumberofPharmacyFirstClinicalPathwaysConsultations-UncomplicatedUTI,NumberofPharmacyFirstUrgentMedicineSupplyConsultations,NumberofPharmacyFirstMinorIllnessReferralConsultations,NumberofPharmacyFirstClinicalPathwaysConsultations-Total
0,East of England,2024-02-01,2353,4347,640,347,414,2175,2965,7814,11740,13241
1,East of England,2024-03-01,3073,6161,646,423,450,2154,3441,8761,11682,16348
2,East of England,2024-04-01,2713,6345,652,683,534,2216,4188,8494,10522,17331
3,East of England,2024-05-01,2642,6413,569,2210,521,1826,4260,10226,11413,18441
4,East of England,2024-06-01,2046,5791,607,2991,551,1712,4362,10228,11761,18060
5,East of England,2024-07-01,2464,6379,615,4427,611,1669,5112,12925,11154,21277
6,London,2024-02-01,1476,4539,475,371,325,1612,3243,14553,23163,12041
7,London,2024-03-01,2221,7423,522,540,359,2053,4078,17962,25178,17196
8,London,2024-04-01,1696,7292,508,634,384,1850,4269,16494,26416,16633
9,London,2024-05-01,2315,9567,595,1810,446,2302,5694,21260,27272,22729


### **TO DO - Compare these against the numbers we get from OpenSAFELY

#### Medication supply data analysis
Data on medications supplied under a Pharmacy First clinical pathway is published by NHS BSA within the [Prescription Cost Analysis (PCA) dataset](https://opendata.nhsbsa.net/dataset/prescription-cost-analysis-pca-monthly-data) - from Feb 24 onwards the PCA dataset contains a column "PHARMACY_ADVANCED_SERVICE" which contains a value 'Pharmacy First Clinical Pathways' where medication was supplied under the Pharmacy First service.

Here we fetch all Pharmacy First medications from PCA data from Feb 24 onwards then combine to return as a single dataframe: